# 3. 데이터 전처리 이해와 실무
## 3.1 데이터 정제 : 이상 데이터 처리
### 이상치(Outlier)
- 관측된 데이터 내 전체적 패턴에서 아주 작게 혹은 아주 크게 벗어난 값  
=> 데이터 분석 과정 및 결과에서 영향을 미쳐 **분석 결과 및 의사 결정의 왜곡**을 불러일으킬 수 있음
### 이상치 유형
- 잘못 수집된 경우 (오류 데이터) : 센서 혹은 시스템 오류로 인해 잘못된 값을 수집
- 실제 극단치의 값 (실제 데이터) : 극단적 관측치로 인해 일반적 관측 범위를 벗어남
### 이상치 처리 프로세스
- 통계적 방안 or 인적 판단 방안 => 이상치 처리
- 통계적 방안의 이상치 판단 : Z-Score, IQR 등
- 인적 판단의 이상치 판단 : 현업 관리자의 데이터 및 비즈니스 관점 기반의 이상치 여부 판단
- 이상치 처리 : 삭제, 대체 등의 방안을 적용하여 처리

#### 이상치 판단 방안
##### Z-Score
- 해당 데이터가 평균으로부터 얼마의 표준편차 만큼 벗어나 있는지를 의미
- 일반적으로, -3이하 3이상 Z값 관측치는 이상치로 정의  
`Z = (x - mean) / std.dev`
##### IQR (Interquartile Range)
- 사분위수 개념 기반의 이상치 판단 방안으로 데이터 흩어짐 정도를 기반으로 이상치를 정의
- Min, Max 범위를 벗어난 관측치는 이상치로 정의
### 이상치 처리 방안
#### 삭제 
- 대체방안이 없거나 이상치 수가 적을 경우 활용
- 이상치 값이 의미를 지니지 않을 경우 활용
- **데이터 손실 발생**
#### 대체 및 변경
- 최대한 데이터 활용을 목적으로 할 경우 활용
- 현업 관점에서 정의된 기준 기반의 다른 값으로 대체
- 데이터가 의미를 지닐 경우 Log 변환 등을 활용한 데이터 간격 조정으로 변경
- **현업/데이터 관점을 신중히 고려하여 적용 필요**

## 실습
샘플 데이터 필요

In [1]:
import numpy as np
import pandas as pd

In [ ]:
click_data = pd.read_csv("./data/click_sample_data.csv", encoding='cp949')
click_data.head(15)

In [ ]:
click_data.info()

click_copy = click_data.copy()

In [ ]:
click_copy['num_click'].describe()

### 이상치 처리
#### Z-Score
- 수식 : `Z = (x-mean) / std.dev`

In [ ]:
click_copy['z_score'] = (click_copy['num_click'] - np.mena(click_copy['num_click'])) / np.std(click_copy['num_click'])
click_copy.head()

In [ ]:
click_copy.describe()

In [ ]:
click_copy = click_copy[ (click_copy['z_score'] < 3) & (click_copy['z_score'] > -3)]

click_copy.info()

In [ ]:
# scipy 내 z-score 함수 제공
from scipy.stats import zscore
click_copy = click_data.copy()

click_copy['z_score'] = zscore(click_copy['num_click'])
click_copy = click_copy[ (click_copy['z_score'] < 3) & (click_copy['z_score'] > -3)]
print(click_copy.shape)

In [ ]:
click_copy.describe()

#### IQR 판단 기반 이상치 처리
click_copy = click_data.copy()

In [ ]:
# 1, 3 분위수 (Q1, Q3) 구하기
q1 = click_copy['num_click'].quantile(0.25)
q3 = click_copy['num_click'].quantile(0.75)

# IQR 구하기
iqr = q3 - q1
iqr

In [ ]:
# IQR 기반 이상치 제거
click_copy = click_copy[ (click_copy['num_click'] > (q1 - 1.5*iqr)) & (click_copy['num_click'] < (q3 + 1.5*iqr))]
click_copy.describe()

In [ ]:
np.shape(click_copy)

### 이상치 대체하기
- 일반적으로 이상치 대체 및 번경은 기존 도메인 지식 및 현업 담당자와의 협의를 통해 진행
- 위 데이터 예제에서 기사 클릭 수의 Max 값을 1000회로 정의하고 1000회 이상의 수 값은 모두 1000회로 대체

In [ ]:
click_copy = click_data.copy()

In [ ]:
# 대체할 기준 정의
max_click = 10000

In [ ]:
click_copy['new_num_click'] = click_copy['num_click']

click_copy.loc[click_copy['new_num_click'] > max_click, 'new_num_click'] = max_click
click_copy.describe()

In [ ]:
click_copy.info()